In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../valid_sentences_filtered.tsv', sep='\t', header=None, names=['e1_id', 'e2_id', 'rel_id', 'e1_label', 'e2_label', 'rel_label', 'sent'])

In [3]:
data

,e1_id,e2_id,rel_id,e1_label,e2_label,rel_label,sent
0,Q275740,Q869,P131,Kanchanaburi,Thailand,located in the administrative territorial entity,Kanchanaburi is the largest of the western pro...
1,Q1083043,Q215380,P31,Christie,band,instance of,Christie is an English rock band that formed a...
2,Q35666,Q23392,P186,glacier,ice,material used,A glacier is a persistent body of dense ice th...
3,Q1238748,Q41298,P31,Domus,magazine,instance of,Domus is an architecture and design magazine f...
4,Q1238748,Q41298,P31,Domus,magazine,instance of,"Published by Editoriale Domus, the magazine is..."
5,Q265,Q232,P47,Uzbekistan,Kazakhstan,shares border with,Uzbekistan is bordered by five landlocked coun...
6,Q159729,Q5372,P641,San Antonio Spurs,basketball,sport,The San Antonio Spurs are an American professi...
7,Q382389,Q28389,P106,Jacques Audiard,screenwriter,occupation,Jacques Audiard is a French film director and ...
8,Q2333456,Q3372216,P161,Spider-Man,Paul Soles,cast member,The show starred the voice of Paul Soles as Pe...
9,Q465282,Q7850,P1412,Liu Xiang,Chinese,"languages spoken, written or signed","Liu Xiang, born Liu Gengsheng and bearing the ..."


### Run Stanford CoreNLP

In [4]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [5]:
openie_outputs = []
len_data = len(data['sent'])
for i in range(len_data):
    print(i, 'of', len_data - 1)
    corenlpres = nlp.annotate(data['sent'][i], properties={
            'ssplit.eolonly': True,
            'openie.triple.all_nominals': False,
            'openie.triple.strict': True,
            'openie.max_entailments_per_clause': 500,
            'annotators': 'openie',
            'outputFormat': 'json',
        })
    for o in corenlpres['sentences'][0]['openie']:
        if ((data['e1_label'][i] in o['subject'] or o['subject'] in data['e1_label'][i])
            and (data['e2_label'][i] in o['object'] or o['object'] in data['e2_label'][i])):
            # subject and object extracted by open IE contains the focus entity
            openie_outputs.append((1, i, o['subject'], o['relation'], o['object']))
        elif ((data['e1_label'][i] not in o['subject'] and o['subject'] not in data['e1_label'][i])
            or (data['e2_label'][i] not in o['object'] and o['object'] not in data['e2_label'][i])):
            openie_outputs.append((0, i, o['subject'], o['relation'], o['object']))

0 of 1398
1 of 1398
2 of 1398
3 of 1398
4 of 1398
5 of 1398
6 of 1398
7 of 1398
8 of 1398
9 of 1398
10 of 1398
11 of 1398
12 of 1398
13 of 1398
14 of 1398
15 of 1398
16 of 1398
17 of 1398
18 of 1398
19 of 1398
20 of 1398
21 of 1398
22 of 1398
23 of 1398
24 of 1398
25 of 1398
26 of 1398
27 of 1398
28 of 1398
29 of 1398
30 of 1398
31 of 1398
32 of 1398
33 of 1398
34 of 1398
35 of 1398
36 of 1398
37 of 1398
38 of 1398
39 of 1398
40 of 1398
41 of 1398
42 of 1398
43 of 1398
44 of 1398
45 of 1398
46 of 1398
47 of 1398
48 of 1398
49 of 1398
50 of 1398
51 of 1398
52 of 1398
53 of 1398
54 of 1398
55 of 1398
56 of 1398
57 of 1398
58 of 1398
59 of 1398
60 of 1398
61 of 1398
62 of 1398
63 of 1398
64 of 1398
65 of 1398
66 of 1398
67 of 1398
68 of 1398
69 of 1398
70 of 1398
71 of 1398
72 of 1398
73 of 1398
74 of 1398
75 of 1398
76 of 1398
77 of 1398
78 of 1398
79 of 1398
80 of 1398
81 of 1398
82 of 1398
83 of 1398
84 of 1398
85 of 1398
86 of 1398
87 of 1398
88 of 1398
89 of 1398
90 of 1398
91 of 139

694 of 1398
695 of 1398
696 of 1398
697 of 1398
698 of 1398
699 of 1398
700 of 1398
701 of 1398
702 of 1398
703 of 1398
704 of 1398
705 of 1398
706 of 1398
707 of 1398
708 of 1398
709 of 1398
710 of 1398
711 of 1398
712 of 1398
713 of 1398
714 of 1398
715 of 1398
716 of 1398
717 of 1398
718 of 1398
719 of 1398
720 of 1398
721 of 1398
722 of 1398
723 of 1398
724 of 1398
725 of 1398
726 of 1398
727 of 1398
728 of 1398
729 of 1398
730 of 1398
731 of 1398
732 of 1398
733 of 1398
734 of 1398
735 of 1398
736 of 1398
737 of 1398
738 of 1398
739 of 1398
740 of 1398
741 of 1398
742 of 1398
743 of 1398
744 of 1398
745 of 1398
746 of 1398
747 of 1398
748 of 1398
749 of 1398
750 of 1398
751 of 1398
752 of 1398
753 of 1398
754 of 1398
755 of 1398
756 of 1398
757 of 1398
758 of 1398
759 of 1398
760 of 1398
761 of 1398
762 of 1398
763 of 1398
764 of 1398
765 of 1398
766 of 1398
767 of 1398
768 of 1398
769 of 1398
770 of 1398
771 of 1398
772 of 1398
773 of 1398
774 of 1398
775 of 1398
776 of 1398
777 

1357 of 1398
1358 of 1398
1359 of 1398
1360 of 1398
1361 of 1398
1362 of 1398
1363 of 1398
1364 of 1398
1365 of 1398
1366 of 1398
1367 of 1398
1368 of 1398
1369 of 1398
1370 of 1398
1371 of 1398
1372 of 1398
1373 of 1398
1374 of 1398
1375 of 1398
1376 of 1398
1377 of 1398
1378 of 1398
1379 of 1398
1380 of 1398
1381 of 1398
1382 of 1398
1383 of 1398
1384 of 1398
1385 of 1398
1386 of 1398
1387 of 1398
1388 of 1398
1389 of 1398
1390 of 1398
1391 of 1398
1392 of 1398
1393 of 1398
1394 of 1398
1395 of 1398
1396 of 1398
1397 of 1398
1398 of 1398


In [6]:
openie_outputs

[(0, 0, 'Kanchanaburi', 'is largest of', 'provinces'),
 (0, 0, 'Kanchanaburi', 'is largest of', 'changwat'),
 (0, 0, 'Kanchanaburi', 'is largest of', 'western provinces'),
 (0, 0, 'Kanchanaburi', 'is', 'largest'),
 (0, 0, 'Thailand', 'of provinces is', 'changwat'),
 (1, 0, 'Kanchanaburi', 'is largest of', 'western provinces of Thailand'),
 (1, 0, 'Kanchanaburi', 'is largest of', 'provinces of Thailand'),
 (0, 1, 'Christie', 'is', 'English'),
 (0, 2, 'accumulation', 'exceeds centuries over', 'years'),
 (0, 2, 'accumulation', 'exceeds centuries over', 'many years'),
 (0, 2, 'its ablation', 'accumulation of', 'snow'),
 (0, 2, 'glacier', 'is', 'persistent'),
 (0, 3, 'Domus', 'founded in', '1928'),
 (1, 3, 'Domus', 'is', 'architecture magazine founded in 1928'),
 (0, 3, 'Domus', 'founded by', 'architect Gio Ponti'),
 (1, 3, 'Domus', 'is', 'architecture magazine founded by architect Gio Ponti'),
 (1, 3, 'Domus', 'is', 'architecture magazine founded'),
 (1,
  3,
  'Domus',
  'is',
  'architec

In [7]:
with open('../valid_openie_filtered.tsv', 'w') as f:
    for item in openie_outputs:
        f.write(str(item[0]) + '\t' + str(item[1]) + '\t' + item[2] + '\t' + item[3] + '\t' + item[4] + '\n')
f.closed

True